In [65]:
# %pip install -qU langchain-community pypdf

# !sudo apt-get install tesseract-ocr tesseract-ocr-fas poppler-utils

# !pip install pytesseract pillow pdf2image

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
import hashlib
import base64
from openai import OpenAI

import os
import json
import fitz
import pytesseract
from PIL import Image

In [3]:
client = OpenAI()

In [83]:
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def extract_text_and_images(pdf_path, output_folder, min_width=100, min_height=100, min_filesize=5000):
    """
    Extract text and unique images from each page of a PDF.
    Skips small icons/logos and duplicate images (by size + dimensions).
    Saves extracted images in output_folder.
    Returns a list of dictionaries (one per page) with the same structure as the OCR version.
    """
    doc = fitz.open(pdf_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    seen = set()  # store (width, height, filesize) triples
    results = []  # list of page dictionaries
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        page_dict = {
            "page": page_num + 1,
            "text": "",
            "screenshots": [],
            "screenshot_texts": []
        }
        
        # --- Extract Text ---
        text = page.get_text("text").strip()
        page_dict["text"] = text if text else ""
        
        # --- Extract Images ---
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            width, height = img[2], img[3]
            
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            filesize = len(image_bytes)
            
            # Skip small images
            if width < min_width or height < min_height or filesize < min_filesize:
                continue
            
            # Skip duplicates
            key = (width, height, filesize)
            if key in seen:
                continue
            seen.add(key)
            
            # Save unique image
            image_output_path = os.path.join(
                output_folder, f"page_{page_num+1}_img_{img_index+1}.{image_ext}"
            )
            with open(image_output_path, "wb") as f:
                f.write(image_bytes)
            
            page_dict["screenshots"].append({
                "path": image_output_path,
            })
        
        results.append(page_dict)
    
    doc.close()
    return results


def extract_text_and_images_ocr(pdf_path, output_folder, min_width=100, min_height=100, min_filesize=5000):
    """
    Extract Persian OCR text and unique images from each page of a PDF.
    Skips small icons/logos and duplicate images (by size + dimensions).
    Saves extracted images in output_folder.
    Returns a list of dictionaries (one per page).
    """
    doc = fitz.open(pdf_path)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    seen = set()  # store (width, height, filesize) triples
    results = []  # list of page dictionaries
    
    for page_num in range(len(doc)):
        print("page:", page_num)
        # if page_num > 2:
        #     break
            
        page = doc.load_page(page_num)
        page_dict = {"page": page_num + 1, "text": "", "screenshots": [], "screenshot_texts": []}
        
        # --- OCR Text (Persian) ---
        # Convert the page to an image for OCR
        pix = page.get_pixmap(dpi=300)
        temp_img_path = os.path.join(output_folder, f"page_{page_num+1}_ocr.png")
        pix.save(temp_img_path)
        page_text = pytesseract.image_to_string(Image.open(temp_img_path), lang="fas")
        page_dict["text"] = page_text.strip()
        
        # --- Extract Images ---
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            width, height = img[2], img[3]
            
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            filesize = len(image_bytes)
            
            # Skip small images
            if width < min_width or height < min_height or filesize < min_filesize:
                continue
            
            # Skip duplicates
            key = (width, height, filesize)
            if key in seen:
                continue
            seen.add(key)
            
            # Save unique image
            image_output_path = os.path.join(
                output_folder, f"page_{page_num+1}_img_{img_index+1}.{image_ext}"
            )
            with open(image_output_path, "wb") as f:
                f.write(image_bytes)
            
            page_dict["screenshots"].append({
                "path": image_output_path,
            })
        
        results.append(page_dict)
    
    doc.close()
    return results


def convert_image_to_text(image_path):
    instruction = (
        "شما یک اسکرین‌شات با حاشیه‌نویسی دریافت کرده‌اید. "
        "تمام حاشیه‌نویسی‌ها، برچسب‌ها، فلش‌ها، هایلایت‌ها و عناصر رابط کاربری قابل مشاهده "
        "(دکمه‌ها، منوها، آیکون‌ها، فیلدهای ورودی و غیره) را بررسی کنید. "
        "تمام اطلاعات مهم را به صورت نکات کوتاه و منظم در قالب بولت پوینت استخراج کنید. "
        "معنای حاشیه‌نویسی‌ها و عملکرد عناصر رابط کاربری را حفظ کنید. "
        "بولت پوینت‌ها را بر اساس بخش‌ها، ویژگی‌ها یا مناطق اسکرین‌شات سازمان‌دهی کنید. "
        "توضیح صرفاً سبک‌های بصری لازم نیست مگر اطلاعات عملکردی داشته باشند. "
        "اگر هیچ اطلاعات قابل استخراجی وجود ندارد، خروجی باید یک رشته خالی ('') باشد."
    )
    
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
    model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": instruction},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_image}"}}
                ],
            }
        ],
    )
    screenshot_text = response.choices[0].message.content
    
    return screenshot_text

In [101]:
pdf_path = "/home/jovyan/projects/data/helpkasra.pdf"
pdf_path = "/home/jovyan/projects/data/dining.pdf"
output_folder = "/home/jovyan/projects/data/extracted_images"

# pdf_data = extract_text_and_images(pdf_path, output_folder)
pdf_data = extract_text_and_images_ocr(pdf_path, output_folder)

page: 0
page: 1
page: 2
page: 3


In [103]:
for page_data in pdf_data:
    print("page:", page_data["page"])
    for screenshot in page_data["screenshots"]:
        screenshot_text = convert_image_to_text(screenshot["path"])
        if len(screenshot_text) > 10:
            page_data["screenshot_texts"].append(screenshot_text)

page: 1
page: 2
page: 3


In [104]:
pdf_data

[{'page': 1,
  'text': 'رزرو غدا\n\nاوت\u200cهلنک\n\nبح\n\n۱۱\n\n۷ /\n۲۲۱ ۳/۸ /',
  'screenshots': [{'path': '/home/jovyan/projects/data/extracted_images/page_1_img_1.jpeg'},
   {'path': '/home/jovyan/projects/data/extracted_images/page_1_img_2.jpeg'}],
  'screenshot_texts': []},
 {'page': 2,
  'text': '۱۱\n۸\n\nدر این داکیومنت یاد میگیریم که:\nچگونه برای پرسنل غذا رزرو کنیم.\n\n۰ صفحه ی رزرو به ما چه اطلاعاتی می دهد.\n\nدر این صفحه می توانیم برای پرسنل و یا کارت\u200cهای متفرقه غذا رزرو کنیم.\n\nبرای دسترسی به این صفحه از طریق منو سامانه\u200cی تغذیه»» رزرو غذا »» رزرو غذا اقدام می\u200cکنیم.\nبسیاری از سازمانهایی که می\u200cخواهند به پرسنل خود حق انتخاب بیش از یک مدل غذا در\nوعده\u200cهای غذایی بدهند از این قابلیت استفاده می\u200cکنند. همچنین این قابلیت باعت میيشه غذا\nفقط به تعداد افرادی که قطعی می\u200cخواهند سرو کنند طبحخ شود و این کار باعت صرفه\u200cجویی در\nمصرف مواد غذایی نیز میشود.\n\nدر این صفحه لبست کل غذاهایی که در برنامه\u200cهای غذایی تعریف شده به شما نمایش داده\nمی\u200c

# test

In [ ]:
# content = [
#     {"type": "text", "text": f"لطفاً این متن و تصاویر همراه آن را بررسی کن و **توضیح کامل بده**، بدون هیچ مقدمه‌ای:\n\n{item['text']}"}
# ]

# for img in item["images"]:
#     b64 = encode_image(img["path"])
#     content.append({
#         "type": "image_url",
#         "image_url": {
#             "url": f"data:image/png;base64,{b64}"
#         }
#     })

# response = client.chat.completions.create(
#     model="gpt-4o-mini",
#     messages=[
#         {"role": "system", "content": "تو یک دستیار هوشمند هستی که متن و تصویر را بررسی و توضیح می‌دهی، بدون هیچ جملهٔ اضافی یا مقدمه."},
#         {"role": "user", "content": content}
#     ]
# )

# print(response.choices[0].message.content)